## Subtask A: Entity and Relationship Extraction


In [3]:
#pip install transformers
#pip install spacy
#pip install nltk
#pip install torch

SyntaxError: invalid syntax (1011913231.py, line 5)

In [ ]:
!python -m spacy download en_core_web_sm

In [4]:
import sqlite3
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy 
import nltk
import requests 
import torch
import pandas as pd
from bs4 import BeautifulSoup
pd.set_option("display.max_rows", 200)

c:\Users\ongai\Desktop\DSA3101\WW-AI-GK\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dbpath = 'data/ecmdatabase.db'
con = sqlite3.connect(f"file:{dbpath}?mode=ro", uri=True)
with con:
    result = con.execute("SELECT item1 from companies WHERE stock_symbol = 'TSLA';")
    tsla_item1 = result.fetchall()[0][0]

In [8]:
tsla_item1

" ITEM 1. BUSINESS \n\nOverview \n\nWe design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products. \n\nOur mission is to accelerate the world&#8217;s transition to sustainable

In [26]:
tsla_item1 = tsla_item1.replace('\n', '')
tsla_item1

" ITEM 1. BUSINESS Overview We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products. Our mission is to accelerate the world&#8217;s transition to sustainable energy. We 

## Data Exploration - Text Analysis

## Named Entity Recognition

### NLTK

In [17]:
## using nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
 
# Step Two: Load Data
print(len(tsla_item1))

# Step Three: Tokenise, find parts of speech and chunk words 

for sent in nltk.sent_tokenize(tsla_item1):
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     if hasattr(chunk, 'label'):
        print(chunk.label(), ' '.join(c[0] for c in chunk))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ongai\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


46161
ORGANIZATION BUSINESS
PERSON Mobile Service
GPE Supercharger
ORGANIZATION Services Automotive
GPE Y
GPE S
ORGANIZATION Cybertruck
PERSON Model
ORGANIZATION SUV
PERSON Model
PERSON Model X
ORGANIZATION SUV
PERSON Model
PERSON Model X
ORGANIZATION SUV
ORGANIZATION Cybertruck
ORGANIZATION Tesla Semi
ORGANIZATION FSD
PERSON Energy
ORGANIZATION Generation
PERSON Storage Energy Storage
GPE Powerwall
GPE Megapack
ORGANIZATION GWh
PERSON Solar
PERSON Energy Offerings
ORGANIZATION PPA
PERSON Technology
ORGANIZATION Automotive Battery
GPE Powertrain
PERSON Model S
PERSON Model X
PERSON Cybertruck
ORGANIZATION Tesla Semi
PERSON Vehicle
ORGANIZATION Control
ORGANIZATION Infotainment
GSP Control
ORGANIZATION Artificial
ORGANIZATION FSD Computer
ORGANIZATION FSD
GPE Optimus
PERSON Energy
ORGANIZATION Generation
PERSON Storage Energy Storage
PERSON Solar
ORGANIZATION Energy Systems
PERSON Solar Roof
ORGANIZATION Powerwall
GPE Design
PERSON Energy
ORGANIZATION Generation
PERSON Solar Roof
PERSON

### Spacy

In [40]:
## Using spacy

# load spacy model
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipe("parser")
nlp.enable_pipe("senter")
# load data
doc = nlp(tsla_item1)

# collect unique labels
unique_labels = {}
for ent in doc.ents:
    if ent.label_ not in unique_labels.keys():
        unique_labels[ent.label_] = []
    unique_labels[ent.label_].append((ent.text, ent.start_char, ent.end_char))


- PERSON:      People, including fictional.
- NORP:        Nationalities or religious or political groups.
- FAC:         Buildings, airports, highways, bridges, etc.
- ORG:         Companies, agencies, institutions, etc.
- GPE:         Countries, cities, states.
- LOC:         Non-GPE locations, mountain ranges, bodies of water.
- PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
- EVENT:       Named hurricanes, battles, wars, sports events, etc.
- WORK_OF_ART: Titles of books, songs, etc.
- LAW:         Named documents made into laws.
- LANGUAGE:    Any named language.
- DATE:        Absolute or relative dates or periods.
- TIME:        Times smaller than a day.
- PERCENT:     Percentage, including ”%“.
- MONEY:       Monetary values, including unit.
- QUANTITY:    Measurements, as of weight or distance.
- ORDINAL:     “first”, “second”, etc.
- CARDINAL:    Numerals that do not fall under another type.

In [41]:
unique_labels.keys()

dict_keys(['ORG', 'PERSON', 'CARDINAL', 'MONEY', 'PRODUCT', 'NORP', 'DATE', 'ORDINAL', 'TIME', 'LOC', 'GPE', 'FAC', 'EVENT', 'LAW', 'PERCENT', 'WORK_OF_ART'])

In [42]:
unique_labels['ORG']

[('Mobile Service', 380, 394),
 ('Our Products and Services Automotive', 1942, 1978),
 ('Full Self-Driving (&', 3066, 3086),
 ('Energy Generation and Storage Energy Storage Products Powerwall',
  3162,
  3225),
 ('Solar Energy Offerings', 3925, 3947),
 ('Powertrain Our', 4680, 4694),
 ('Model S', 5321, 5328),
 ('Vehicle Control', 5785, 5800),
 ('Control systems', 5942, 5957),
 ('Self-Driving Development and Artificial Intelligence We', 6250, 6305),
 ('FSD Computer', 6449, 6461),
 ('Autopilot', 6804, 6813),
 ('Tesla', 7312, 7317),
 ('AI', 7652, 7654),
 ('Energy Generation and Storage Energy Storage Products', 7663, 7716),
 ('Solar Energy Systems', 8235, 8255),
 ('Energy Generation and Storage', 9642, 9671),
 ('Megapack', 9945, 9953),
 ('Automotive Direct Sales Our', 10536, 10563),
 ('Tesla', 11052, 11057),
 ('Tesla', 11508, 11513),
 ('Tesla', 11585, 11590),
 ('Tesla', 11737, 11742),
 ('Tesla Superchargers', 11857, 11876),
 ('the North American Charging Standard', 12677, 12713),
 ('NACS'

In [44]:
unique_labels['PRODUCT'] #products

[('the Model 3', 2045, 2056),
 ('Model 3', 2082, 2089),
 ('Model 3', 2277, 2284),
 ('Model S', 2437, 2444),
 ('Solar Roof', 4355, 4365),
 ('Solar Roof', 8275, 8285),
 ('Solar Roof', 10158, 10168),
 ('Solar Roof', 16095, 16105),
 ('Solar Roof', 17592, 17602),
 ('Model 3', 32041, 32048),
 ('Model Y', 32053, 32060)]

In [45]:
unique_labels['LOC'] # locations

[('North America', 16709, 16722),
 ('Europe', 16724, 16730),
 ('Asia', 16735, 16739),
 ('Northern California', 17880, 17899),
 ('Europe', 32867, 32873),
 ('Europe', 41224, 41230)]

In [46]:
unique_labels['FAC'] # facilities / factories

[('Gigafactory Texas', 17963, 17980),
 ('Fremont Factory', 42324, 42339),
 ('Gigafactory Texas', 42361, 42378)]

In [47]:
unique_labels['EVENT'] # events

[('this Annual Report on Form 10-K', 20814, 20845),
 ('this Annual Report on Form 10-K. Energy Storage System Incentives',
  21988,
  22053),
 ('this Annual Report on Form 10-K. Pursuant', 23163, 23204),
 ('this Annual Report on Form 10-K.', 45826, 45858)]

### Hugging Face Transformers

In [9]:
## Using transformers
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_pipe = pipeline("ner", model = model, tokenizer = tokenizer)
for ent in ner_pipe(tsla_item1):
    print(ent)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-ORG', 'score': 0.54775363, 'index': 74, 'word': 'Service', 'start': 391, 'end': 398}
{'entity': 'B-MISC', 'score': 0.5205288, 'index': 84, 'word': 'Des', 'start': 438, 'end': 441}
{'entity': 'I-MISC', 'score': 0.62265116, 'index': 87, 'word': 'Chargers', 'start': 450, 'end': 458}
{'entity': 'B-ORG', 'score': 0.75424445, 'index': 350, 'word': 'Products', 'start': 1960, 'end': 1968}
{'entity': 'I-ORG', 'score': 0.9869141, 'index': 351, 'word': 'and', 'start': 1969, 'end': 1972}
{'entity': 'I-ORG', 'score': 0.9261165, 'index': 352, 'word': 'Services', 'start': 1973, 'end': 1981}
{'entity': 'I-ORG', 'score': 0.88983554, 'index': 353, 'word': 'Auto', 'start': 1984, 'end': 1988}
{'entity': 'I-ORG', 'score': 0.60575384, 'index': 354, 'word': '##mot', 'start': 1988, 'end': 1991}
{'entity': 'B-MISC', 'score': 0.59626323, 'index': 369, 'word': 'Model', 'start': 2067, 'end': 2072}
{'entity': 'I-MISC', 'score': 0.9495684, 'index': 370, 'word': '3', 'start': 2073, 'end': 2074}
